# 🔬 ANÁLISE TEMPORAL DE MUNICIPIOS - PROJETO CAIPORA

<div align="center">
    <img src="image.png">
</div>

## 0. Pré requisitos para funcionamento das respectivas partes

### Importações de bibliotecas

In [ ]:
from ipywidgets import Dropdown, VBox, Button, Output, Text
from IPython.display import display
from pysus.online_data import IBGE
from pysus import SIH
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import geobr
import os

from merradownload.merra_scraping import baixar_merra

In [38]:
codigos_ibge_df = pd.read_csv(r"../data/codigos_ibge.csv", sep=";")
estados = sorted(codigos_ibge_df["uf"].unique())
municipios_por_estado = {
    uf: sorted(codigos_ibge_df[codigos_ibge_df["uf"] == uf]["municipio"].unique())
    for uf in estados
}

estados_dd = Dropdown(options=estados, description='Estado:')
municipios_dd = Dropdown(options=municipios_por_estado[estados[0]], description='Município:')
baixar_btn = Button(description="Baixar e Plotar Shapefile")
output_mapa = Output()

def atualiza_municipios(change):
    uf_selecionado = change['new']
    municipios_dd.options = municipios_por_estado[uf_selecionado]

estados_dd.observe(atualiza_municipios, names='value')

def baixar_plotar_shapefile(b):
    uf = estados_dd.value
    municipio = municipios_dd.value

    localizar_municipio = geobr.lookup_muni(name_muni=municipio)
    localizar_municipio = localizar_municipio[localizar_municipio['abbrev_state'] == uf]

    output_mapa.clear_output()
    
    with output_mapa:
        print(f"Baixando shapefile para {municipio}, {uf}...")
        gdf = geobr.read_municipality(code_muni=localizar_municipio.iloc[0]['code_muni'])
        print("Download concluído.")

        save_dir = rf"../data/shapefiles/{uf}-{municipio.replace(' ', '_')}/"
        os.makedirs(save_dir, exist_ok=True)
        
        # Salva o shapefile
        filepath = os.path.join(save_dir, f"{uf}_{municipio.replace(' ', '_')}.shp")
        gdf.to_file(filepath, driver='ESRI Shapefile')
        print(f"Shapefile salvo em: {filepath}")

        fig, ax = plt.subplots(1, 1, figsize=(10, 10))
        gdf.plot(ax=ax, facecolor='#add8e6', edgecolor='black')
        ax.set_title(f"Mapa de {municipio} - {uf}")
        ax.set_xlabel("Longitude")
        ax.set_ylabel("Latitude")
        plt.grid(True)
        plt.show()

baixar_btn.on_click(baixar_plotar_shapefile)

display(VBox([estados_dd, municipios_dd, baixar_btn, output_mapa]))

## IBGE

### Dados de residentes população

In [ ]:
codigos_ibge = gpd.read_csv(r"data/codigos_ibge.csv", sep=";")

ag = IBGE.list_agregados()

pd.DataFrame(ag[ag.id=='CM'].agregados.iloc[0])

Fetching Data groupings from https://servicodados.ibge.gov.br/api/v3/agregados?


,id,nome
0,305,População residente em domicílios particulares...
1,319,Média de moradores por domicílio particular pe...
2,472,"População residente por idade, forma de declar..."
3,473,Pessoas de 4 anos ou mais de idade por grupos ...
4,475,"População residente por grupos de idade, sexo ..."
5,476,Pessoas de 4 anos ou mais de idade por frequên...
6,477,Pessoas de 4 anos ou mais que frequentam escol...
7,478,Pessoas não residentes no município de residên...
8,479,Chefes de domicílios particulares permanentes ...
9,480,População residente por relação com o chefe do...


In [5]:
opts= [(r.nome, int(r.id)) for r in pd.DataFrame(ag[ag.id=='CM'].agregados.iloc[0]).itertuples()]
ds = Dropdown(
    options=opts,
    value=798,
    description='Number:',
)
display(ds)
IBGE.metadados(ds.value)

Dropdown(description='Number:', index=20, options=(('População residente em domicílios particulares permanente…

{'id': 798,
 'nome': 'Domicílios particulares ocupados por sexo da pessoa responsável pelo domicílio e situação do domicílio',
 'URL': 'https://sidra.ibge.gov.br/tabela/798',
 'pesquisa': 'Contagem da População ',
 'assunto': 'Domicílios',
 'periodicidade': {'frequencia': 'anual', 'inicio': 2007, 'fim': 2007},
 'nivelTerritorial': {'Administrativo': ['N1', 'N2', 'N8', 'N9', 'N6', 'N3'],
  'Especial': [],
  'IBGE': []},
 'variaveis': [{'id': 2048,
   'nome': 'Domicílios particulares ocupados',
   'unidade': 'Unidades',
   'sumarizacao': []},
  {'id': 1002048,
   'nome': 'Domicílios particulares ocupados - percentual do total geral',
   'unidade': '%',
   'sumarizacao': []}],
 'classificacoes': [{'id': 11561,
   'nome': 'Sexo da pessoa responsável pelo domicílio',
   'sumarizacao': {'status': True, 'excecao': []},
   'categorias': [{'id': 0, 'nome': 'Total', 'unidade': None, 'nivel': 0},
    {'id': 100680, 'nome': 'Homens', 'unidade': None, 'nivel': 1},
    {'id': 100681, 'nome': 'Mulher

In [9]:
sih = SIH().load()
 
sih.groups



{'RD': 'AIH Reduzida',
 'RJ': 'AIH Rejeitada',
 'ER': 'AIH Rejeitada com erro',
 'SP': 'Serviços Profissionais',
 'CH': 'Cadastro Hospitalar',
 'CM': ''}

In [3]:
codigos_ibge_df = pd.read_csv(r"../data/codigos_ibge.csv", sep=";")

estados = sorted(codigos_ibge_df["uf"].unique())
municipios_por_estado = {
    uf: sorted(codigos_ibge_df[codigos_ibge_df["uf"] == uf]["municipio"].unique())
    for uf in estados
}

estados_dd = Dropdown(
    options=estados,
    description='Estados:',
    disabled=False
)

municipios_dd = Dropdown(
    options=municipios_por_estado[estados[0]],
    description='Municipios:',
    disabled=False
)

def atualiza_municipios(change):
    uf_selecionado = change['new']
    municipios_dd.options = municipios_por_estado[uf_selecionado]

estados_dd.observe(atualiza_municipios, names='value')

In [4]:
# Widgets para login e senha
login_txt = Text(description='Login:', placeholder='Usuário NASA')
senha_txt = Text(description='Senha:', placeholder='Senha NASA', style={'description_width': 'initial'}, layout={'width': '300px'},  password=True)

baixar_btn = Button(description="Baixar dados da NASA")
output = Output()

def baixar_dados_callback(b):
    municipio = municipios_dd.value
    uf = estados_dd.value
    dados_municipio = codigos_ibge_df[(codigos_ibge_df["uf"] == uf) & (codigos_ibge_df["municipio"] == municipio)]
    if not dados_municipio.empty:
        lat = float(dados_municipio["latitude"].values[0])
        lon = float(dados_municipio["longitude"].values[0])
        nome = municipio
        usuario = login_txt.value
        senha = senha_txt.value
        output.clear_output()
        with output:
            print(f"Baixando dados para {nome} ({lat}, {lon})...")
            baixar_merra(
                username=usuario,
                password=senha,
                years=[2020, 2021],
                field_id="T2M",
                field_name="temperatura",
                database_name="M2I1NXASM",
                database_id="inst1_2d_asm_Nx",
                locs=[(nome, lat, lon)],
                conversion_function=lambda x: x - 273.15,
                aggregator="mean"
            )
            print("Download finalizado!")
    else:
        output.clear_output()
        with output:
            print("Município não encontrado ou sem coordenadas.")

baixar_btn.on_click(baixar_dados_callback)

display(VBox([login_txt, senha_txt, estados_dd, municipios_dd, baixar_btn, output]))